<h1>Experiment 2</h1>
<h3>Test the robustness of each network to the undersampled data</h3>

In [ ]:
import nibabel as nib
import numpy as np

from FormatData import generate_data, parser as data_parser
from Training import train_network
from utils.model import parser as model_parser
from Testing import test_model
from utils.nii_utils import mask_nii_data

In [ ]:
train_subject = 's01_still'
# test_subjects = ['s02_still', 's03_still_reg', 's04_still_reg']
test_subjects = ['s02_still']

In [ ]:
class Namespace:
    """
    a class generate parser for cmd line args
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [ ]:
"""
packages that handle graphs
"""
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, LinearSegmentedColormap
from skimage.metrics import structural_similarity as compare_ssim
from utils import calc_ssim
%matplotlib inline
def plot_loss(cmd):
    """
    A function that used to plot the loss curve for the trained network.
    Args:
        cmd: String, the command line in the terminal
    """
    args = model_parser().parse_args(cmd.split())
    history = train_network(args)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

def show_slices(slices, grayscale=True):
    """
    Function to display the slices

    Args:
        slices (list): a list of 2d ndarray that contains the data to be displayed
        grayscale (bool, optional): True, if diplay grayscale img. Defaults to True.
    """    
    fig, axes = plt.subplots(1, len(slices), figsize=(10,10))
    cax = fig.add_axes([0, 0, .3, .3])
    for i, slice in enumerate(slices):
        # use grayscale for displaying ref and pred imgs:
        if grayscale:
            cmap = plt.get_cmap('gray')
            cmaplist = [cmap(i) for i in range(cmap.N)]
            cmap = LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
            bounds = np.arange(0, 1.0, .01)
            idx = np.searchsorted(bounds, 0)
            bounds = np.insert(bounds, idx, 0)
            norm = BoundaryNorm(bounds, cmap.N)
            im = axes[i].imshow(slice.T, cmap=cmap, origin="lower", interpolation='none', norm=norm)
        else:
            # define the colormap
            cmap = plt.get_cmap('bwr')
            # extract all colors from the .jet map
            cmaplist = [cmap(i) for i in range(cmap.N)]
            # create the new map
            cmap = LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
            # define the bins and normalize and forcing 0 to be part of the colorbar
            # define the min and max to be -1 and +1 respectively
            bounds = np.arange(-0.5, 0.5, .01)
            idx = np.searchsorted(bounds, 0)
            bounds = np.insert(bounds, idx, 0)
            norm = BoundaryNorm(bounds, cmap.N)
            im = axes[i].imshow(slice.T, cmap=cmap, origin="lower", interpolation='none', norm=norm)
        fig.colorbar(im, cax=cax, orientation='vertical')

def scale(img):
    # for i in range(img.shape[0]):
    #     for j in range(img.shape[1]):
    #         img[i][j] = ((img[i][j]+1)/2)*255
    return img

def compare_simi(pred, ref):
    return calc_ssim(pred, ref)

In [ ]:
def visualise0(ref):
    """
    Function to visualise the ref imgs

    Args:
        ref (ndarray): the reference data
    """
    # visualise the ref imgs
    ref0 = ref[26, :, :]
    ref1 = ref[:, 30, :]
    ref2 = ref[:, :, 16]
    show_slices([ref0, ref1, ref2])
    plt.suptitle("Center slices for reference image")


def visualise1(ref_ndi, retained_vol, subject, model, layer, affine):
    """
    Function to visualise the NDI imgs and store the difference map

    Args:
        ref_ndi (ndarray): the reference NDI data
        subject (string): the subject that is examined
        model (string): the model used
        layer (int): the number of layers for the network
        retained_vol (int): the number of volumes used
    """
    patch = 3
    if model == 'fc1d':
        patch = 1

    print(patch)
    print(model)
    # visualise the ref imgs
    refNDI0 = ref_ndi[26, :, :]
    refNDI1 = ref_ndi[:, 30, :]
    refNDI2 = ref_ndi[:, :, 16]

    # visualise the pred imgs produced at varied input size
    ndi_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_NDIsynthetic.nii'
    ndi_img = nib.load(ndi_path)
    ndi_data = ndi_img.get_fdata()

    ndi0 = ndi_data[26, :, :]
    ndi1 = ndi_data[:, 30, :]
    ndi2 = ndi_data[:, :, 16]
    show_slices([ndi0, ndi1, ndi2])
    plt.suptitle('Center slices for NDI predicted image by '+model+', input size='+str(retained_vol))
    (score, ndidiff) = compare_ssim(ndi_data, ref_ndi, full=True)
    print(str(retained_vol)+'input size the ssim score for ndi is: ' + str(score))

    # plot the difference map between the imgs by the lib
    ndidiff0 = scale(refNDI0 - ndi0)
    ndidiff1 = scale(refNDI1 - ndi1)
    ndidiff2 = scale(refNDI2 - ndi2)
    show_slices([ndidiff0, ndidiff1, ndidiff2], grayscale=False)
    plt.suptitle("Difference map NDI")

    diff_img_np = ref_ndi - ndi_data
    diff_img = nib.Nifti1Image(diff_img_np, affine)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_NDI_difference_synthetic.nii')

def visualise2(ref_odi, retained_vol, subject, model, layer, affine):
    """
    Function to visualise the ODI imgs and difference maps

    Args:
        ref_odi (ndarray): the reference ODI data
        retained_vol (int): the number of volumes retained after rejection
        subject (string): the subject that is examined
        model (string): the model used
        layer (int): the number of layers for the network
    """
    patch = 3
    if model == 'fc1d':
        patch = 1

    print(patch)
    print(model)
    # visualise the ref imgs
    refODI0 = ref_odi[26, :, :]
    refODI1 = ref_odi[:, 30, :]
    refODI2 = ref_odi[:, :, 16]

    # visualise the pred imgs produced at varied input size
    odi_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_ODIsynthetic.nii'
    odi_img = nib.load(odi_path)
    odi_data = odi_img.get_fdata()

    odi0 = odi_data[26, :, :]
    odi1 = odi_data[:, 30, :]
    odi2 = odi_data[:, :, 16]
    show_slices([odi0, odi1, odi2])
    plt.suptitle('Center slices for ODI predicted image by '+model+', input size='+str(retained_vol))
    (score, odidiff) = compare_ssim(odi_data, ref_odi, full=True)
    print(str(retained_vol)+'input size the ssim score for odi is: ' + str(score))

    odidiff0 = scale(refODI0 - odi0)
    odidiff1 = scale(refODI1 - odi1)
    odidiff2 = scale(refODI2 - odi2)
    show_slices([odidiff0, odidiff1, odidiff2], grayscale=False)
    plt.suptitle("Difference map ODI")

    diff_img_np = ref_odi - odi_data
    diff_img = nib.Nifti1Image(diff_img_np, affine)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_ODI_difference_synthetic.nii')

def visualise3(ref_fwf, retained_vol, subject, model, layer, affine):
    """
    Function to visualise the fwf imgs and difference maps

    Args:
        ref_fwf (ndarray): the reference FWF data
        retained_vol (int): the number of volumes retained after rejection
        subject (string): the subject that is examined
        model (string): the model used
        layer (int): the number of layers for the network
    """
    patch = 3
    if model == 'fc1d':
        patch = 1

    print(patch)
    print(model)
    # visualise the ref imgs
    refFWF0 = ref_fwf[26, :, :]
    refFWF1 = ref_fwf[:, 30, :]
    refFWF2 = ref_fwf[:, :, 16]

    # visualise the pred imgs produced at varied input size
    fwf_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_FWFsynthetic.nii'
    fwf_img = nib.load(fwf_path)
    fwf_data = fwf_img.get_fdata()

    fwf0 = fwf_data[26, :, :]
    fwf1 = fwf_data[:, 30, :]
    fwf2 = fwf_data[:, :, 16]
    show_slices([fwf0, fwf1, fwf2])
    plt.suptitle('Center slices for FWF predicted image by '+model+', input size='+str(retained_vol))
    (score, fwfdiff) = compare_ssim(fwf_data, ref_fwf, full=True)
    print(str(retained_vol)+'input size the ssim score for fwf is: ' + str(score))

    # plot the difference map between the imgs by the lib
    fwfdiff0 = scale(refFWF0 - fwf0)
    fwfdiff1 = scale(refFWF1 - fwf1)
    fwfdiff2 = scale(refFWF2 - fwf2)
    show_slices([fwfdiff0, fwfdiff1, fwfdiff2], grayscale=False)
    plt.suptitle("Difference map FWF")

    diff_img_np = ref_fwf - fwf_data
    diff_img = nib.Nifti1Image(diff_img_np, affine)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_FWF_difference_synthetic.nii')

def visualise4(ref_ndi, ref_odi, ref_fwf, retained_vol, subject, model, layer, affine1, affine2, affine3):
    """
    Function to visualise the imgs and difference maps

    Args:
        ref_ndi (ndarray): the reference NDI data
        ref_odi (ndarray): the reference ODI data
        ref_fwf (ndarray): the reference FWF data
        retained_vol (int): the number of volumes retained after rejection
        subject (string): the subject that is examined
        model (string): the model used
        layer (int): the number of layers for the network
    """
    patch = 3
    if model == 'fc1d':
        patch = 1

    print(patch)
    print(model)
    # visualise the ref imgs
    refNDI0 = ref_ndi[26, :, :]
    refNDI1 = ref_ndi[:, 30, :]
    refNDI2 = ref_ndi[:, :, 16]

    refODI0 = ref_odi[26, :, :]
    refODI1 = ref_odi[:, 30, :]
    refODI2 = ref_odi[:, :, 16]

    refFWF0 = ref_fwf[26, :, :]
    refFWF1 = ref_fwf[:, 30, :]
    refFWF2 = ref_fwf[:, :, 16]

    # visualise the pred imgs produced at varied input size
    ndi_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_NDIsynthetic.nii'
    odi_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_ODIsynthetic.nii'
    fwf_path = '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_FWFsynthetic.nii'
    ndi_img = nib.load(ndi_path)
    ndi_data = ndi_img.get_fdata()
    odi_img = nib.load(odi_path)
    odi_data = odi_img.get_fdata()
    fwf_img = nib.load(fwf_path)
    fwf_data = fwf_img.get_fdata()

    ndi0 = ndi_data[26, :, :]
    ndi1 = ndi_data[:, 30, :]
    ndi2 = ndi_data[:, :, 16]
    show_slices([ndi0, ndi1, ndi2])
    plt.suptitle('Center slices for NDI predicted image by '+model+', input size='+str(retained_vol))
    (score, ndidiff) = compare_ssim(ndi_data, ref_ndi, full=True)
    print(str(retained_vol)+'input size the ssim score for ndi is: ' + str(score))

    odi0 = odi_data[26, :, :]
    odi1 = odi_data[:, 30, :]
    odi2 = odi_data[:, :, 16]
    show_slices([odi0, odi1, odi2])
    plt.suptitle('Center slices for ODI predicted image by '+model+', input size='+str(retained_vol))
    (score, odidiff) = compare_ssim(odi_data, ref_odi, full=True)
    print(str(retained_vol)+'input size the ssim score for odi is: ' + str(score))

    fwf0 = fwf_data[26, :, :]
    fwf1 = fwf_data[:, 30, :]
    fwf2 = fwf_data[:, :, 16]
    show_slices([fwf0, fwf1, fwf2])
    plt.suptitle('Center slices for FWF predicted image by '+model+', input size='+str(retained_vol))
    (score, fwfdiff) = compare_ssim(fwf_data, ref_fwf, full=True)
    print(str(retained_vol)+'input size the ssim score for fwf is: ' + str(score))

    # plot the difference map between the imgs by the lib
    ndidiff0 = scale(refNDI0 - ndi0)
    ndidiff1 = scale(refNDI1 - ndi1)
    ndidiff2 = scale(refNDI2 - ndi2)
    show_slices([ndidiff0, ndidiff1, ndidiff2], grayscale=False)
    plt.suptitle("Difference map NDI")

    diff_img_np = ref_ndi - ndi_data
    diff_img = nib.Nifti1Image(diff_img_np, affine1)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_NDI_difference_synthetic.nii')

    odidiff0 = scale(refODI0 - odi0)
    odidiff1 = scale(refODI1 - odi1)
    odidiff2 = scale(refODI2 - odi2)
    show_slices([odidiff0, odidiff1, odidiff2], grayscale=False)
    plt.suptitle("Difference map ODI")

    diff_img_np = ref_odi - odi_data
    diff_img = nib.Nifti1Image(diff_img_np, affine2)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_ODI_difference_synthetic.nii')

    fwfdiff0 = scale(refFWF0 - fwf0)
    fwfdiff1 = scale(refFWF1 - fwf1)
    fwfdiff2 = scale(refFWF2 - fwf2)
    show_slices([fwfdiff0, fwfdiff1, fwfdiff2], grayscale=False)
    plt.suptitle("Difference map FWF")

    diff_img_np = ref_fwf - fwf_data
    diff_img = nib.Nifti1Image(diff_img_np, affine3)
    nib.save(diff_img, '../Net/nii/'+subject+'-'+str(retained_vol)+'-'+model+'-patch_'+str(patch)+'-base_1-layer_'+str(layer)+'-label_FWF_difference_synthetic.nii')

___
<h3>Data Preprocessing</h3>

In [ ]:
# motion-free subject path
s01_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/'
s02_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/'
s03_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/'
s04_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/'
# motion-free target labels
s01_NDI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/s01_still_NDI.nii'
s02_NDI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/s02_still_NDI.nii'
s03_NDI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/s03_still_reg_NDI.nii'
s04_NDI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/s04_still_reg_NDI.nii'

s01_ODI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/s01_still_ODI.nii'
s02_ODI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/s02_still_ODI.nii'
s03_ODI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/s03_still_reg_ODI.nii'
s04_ODI_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/s04_still_reg_ODI.nii'

s01_FWF_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/s01_still_FWF.nii'
s02_FWF_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/s02_still_FWF.nii'
s03_FWF_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/s03_still_reg_FWF.nii'
s04_FWF_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/s04_still_reg_FWF.nii'

In [ ]:
def filter_mask(subpath, fwfpath, threshold=0.99):
    """
    By looking at the imgs generated, we have found out there are some regions that should not be included. Since they have values higher than 1.0
    And we have found out voxels have NDI and ODI values, while that voxel has GROUND TRUTH FWF 1.0
    This should indicate that that voxel should not even be included in the training
    Therefore we want to filter the each subject's mask first, by using their corresponding GROUND TRUTH FWF

    Args:
        subpath (string): the path of the subject folder
        fwfpath (string): the path of the corresponding fwf file
        threshold (float): the thresholds to be used to filter of the mask,
                           a stringnent threshold would be 0.9, the least stringnent threshold is 1.0
                           by default, it is set to 0.99
    """
    # fetch the mask data
    img_mask = nib.load(subpath+'mask-e.nii')
    original_mask = img_mask.get_fdata()
    original_affine = img_mask.affine
    shape = original_mask.shape # retain the shape of the mask
    origin_nonzeros = np.count_nonzero(original_mask)
    print('original mask has: ' + str(origin_nonzeros) + ' of nonzero voxels')
    # fetch the FWF data
    fwf = nib.load(fwfpath).get_fdata()
    # filter
    mask = original_mask.flatten() # this makes a copy of the orginal mask
    fwf = fwf.reshape(mask.shape[0]) # reshape fwf to the corresponding shape
    for i in range(len(mask)):
        # if fwf has high value, means there is no tissue
        # therefore, the voxel should be excluded
        if fwf[i] >= threshold:
            mask[i] = 0.0
    # reshape mask back
    mask = mask.reshape(shape)
    filter_nonzeros = np.count_nonzero(mask)
    print('filtered mask has: ' +str(filter_nonzeros) + ' of nonzero voxels')
    # save the mask
    filter_img = nib.Nifti1Image(mask, original_affine)
    nib.save(filter_img, subpath+'filtered_mask.nii')

In [ ]:
# Use the above code to filter each subject's mask. Store as filtered_mask.nii in each subject folder
filter_mask(s01_path, s01_FWF_path)
filter_mask(s02_path, s02_FWF_path)
filter_mask(s03_path, s04_FWF_path)
filter_mask(s04_path, s03_FWF_path)

In [ ]:
# filtered mask path for each subject
s01_mask_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s01_still/filtered_mask.nii'
s02_mask_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s02_still/filtered_mask.nii'
s03_mask_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s03_still_reg/filtered_mask.nii'
s04_mask_path = '/home/vw/Desktop/IndividualProject/MedICSS2021_/Data-NODDI/s04_still_reg/filtered_mask.nii'

In [ ]:
"""
Generate base datasets for each subject
"""
ltype = ['A']
for l in ltype:
    print('Generating basedataset for label: ' + l)
    cmd = '--base --label_type ' + l + ' --subjects s01_still s02_still'
    args = data_parser().parse_args(cmd.split())
    generate_data(args)

In [ ]:
"""
Generate training dataset for ANN.
"""
cmd = "--subjects s01_still s02_still --fc1d --label_type A"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
# Generate the dataset for 2D CNN
cmd = "--subjects s01_still s02_still --conv2d --label_type A"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
# Generate the dataset for 3D CNN
cmd = "--subjects s01_still s02_still --conv3d --label_type A"
args = data_parser().parse_args(cmd.split())
generate_data(args)

In [ ]:
"""
Using nib to fetch the ground truth img
"""
#load the truth data for subject 1
s01_NDI_img = nib.load(s01_NDI_path)
s01_ODI_img = nib.load(s01_ODI_path)
s01_FWF_img = nib.load(s01_FWF_path)
s01_NDI_affine = s01_NDI_img.affine
s01_ODI_affine = s01_ODI_img.affine
s01_FWF_affine = s01_FWF_img.affine
s01_NDI_img_data = s01_NDI_img.get_fdata()
s01_ODI_img_data = s01_ODI_img.get_fdata()
s01_FWF_img_data = s01_FWF_img.get_fdata()
#load the truth data for subject 2
s02_NDI_img = nib.load(s02_NDI_path)
s02_ODI_img = nib.load(s02_ODI_path)
s02_FWF_img = nib.load(s02_FWF_path)
s02_NDI_affine = s02_NDI_img.affine
s02_ODI_affine = s02_ODI_img.affine
s02_FWF_affine = s02_FWF_img.affine
s02_NDI_img_data = s02_NDI_img.get_fdata()
s02_ODI_img_data = s02_ODI_img.get_fdata()
s02_FWF_img_data = s02_FWF_img.get_fdata()
# load the truth data for subject 3
s03_NDI_img = nib.load(s03_NDI_path)
s03_ODI_img = nib.load(s03_ODI_path)
s03_FWF_img = nib.load(s03_FWF_path)
s03_NDI_affine = s03_NDI_img.affine
s03_ODI_affine = s03_ODI_img.affine
s03_FWF_affine = s03_FWF_img.affine
s03_NDI_img_data = s03_NDI_img.get_fdata()
s03_ODI_img_data = s03_ODI_img.get_fdata()
s03_FWF_img_data = s03_FWF_img.get_fdata()
# load the truth data for subject 4
s04_NDI_img = nib.load(s04_NDI_path)
s04_ODI_img = nib.load(s04_ODI_path)
s04_FWF_img = nib.load(s04_FWF_path)
s04_NDI_affine = s04_NDI_img.affine
s04_ODI_affine = s04_ODI_img.affine
s04_FWF_affine = s04_FWF_img.affine
s04_NDI_img_data = s04_NDI_img.get_fdata()
s04_ODI_img_data = s04_ODI_img.get_fdata()
s04_FWF_img_data = s04_FWF_img.get_fdata()

<h4>Test the performace of each network to generate each parameter with varied number of DWI as input size</h4>

Each network (ANN, 2D CNN and 3D CNN) should be implemented with 5 hidden layers. The choice is suggested from the obtained results from the previous experiment

DWI = 4

In [ ]:
dwi = 4

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 5

In [ ]:
dwi = 5

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 6

In [ ]:
dwi = 6

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 7

In [ ]:
dwi = 7

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 8

In [ ]:
dwi = 8

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 10

In [ ]:
dwi = 10

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 12

In [ ]:
dwi = 12

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 14

In [ ]:
dwi = 14

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 16

In [ ]:
dwi = 16

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D 4
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 18

In [ ]:
dwi = 18

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s01_NDI_img_data)
visualise0(s01_ODI_img_data)
visualise0(s01_FWF_img_data)
visualise4(s01_NDI_img_data, s01_ODI_img_data, s01_FWF_img_data, dwi, 's01_still', 'fc1d', 4, s01_NDI_affine, s01_ODI_affine, s01_FWF_affine)
visualise4(s01_NDI_img_data, s01_ODI_img_data, s01_FWF_img_data, dwi, 's01_still', 'conv2d', 4, s01_NDI_affine, s01_ODI_affine, s01_FWF_affine)
visualise4(s01_NDI_img_data, s01_ODI_img_data, s01_FWF_img_data, dwi, 's01_still', 'conv3d', 4, s01_NDI_affine, s01_ODI_affine, s01_FWF_affine)

DWI = 20

In [ ]:
dwi = 20

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s01_NDI_img_data)
visualise0(s01_ODI_img_data)
visualise0(s01_FWF_img_data)
visualise4(s01_NDI_img_data, s01_ODI_img_data, s01_FWF_img_data, dwi, 's01_still', 'fc1d', 4, s01_NDI_affine, s01_ODI_affine, s01_FWF_affine)
visualise4(s01_NDI_img_data, s01_ODI_img_data, s01_FWF_img_data, dwi, 's01_still', 'conv2d', 4, s01_NDI_affine, s01_ODI_affine, s01_FWF_affine)
visualise4(s01_NDI_img_data, s01_ODI_img_data, s01_FWF_img_data, dwi, 's01_still', 'conv3d', 4, s01_NDI_affine, s01_ODI_affine, s01_FWF_affine)

DWI = 22

In [ ]:
dwi = 22

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s01_NDI_img_data)
visualise0(s01_ODI_img_data)
visualise0(s01_FWF_img_data)
visualise4(s01_NDI_img_data, s01_ODI_img_data, s01_FWF_img_data, dwi, 's01_still', 'fc1d', 4, s01_NDI_affine, s01_ODI_affine, s01_FWF_affine)
visualise4(s01_NDI_img_data, s01_ODI_img_data, s01_FWF_img_data, dwi, 's01_still', 'conv2d', 4, s01_NDI_affine, s01_ODI_affine, s01_FWF_affine)
visualise4(s01_NDI_img_data, s01_ODI_img_data, s01_FWF_img_data, dwi, 's01_still', 'conv3d', 4, s01_NDI_affine, s01_ODI_affine, s01_FWF_affine)

DWI = 24

In [ ]:
dwi = 24

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s01_NDI_img_data)
visualise0(s01_ODI_img_data)
visualise0(s01_FWF_img_data)
visualise4(s01_NDI_img_data, s01_ODI_img_data, s01_FWF_img_data, dwi, 's01_still', 'fc1d', 4, s01_NDI_affine, s01_ODI_affine, s01_FWF_affine)
visualise4(s01_NDI_img_data, s01_ODI_img_data, s01_FWF_img_data, dwi, 's01_still', 'conv2d', 4, s01_NDI_affine, s01_ODI_affine, s01_FWF_affine)
visualise4(s01_NDI_img_data, s01_ODI_img_data, s01_FWF_img_data, dwi, 's01_still', 'conv3d', 4, s01_NDI_affine, s01_ODI_affine, s01_FWF_affine)

DWI = 26

In [ ]:
dwi = 26

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 36

In [ ]:
dwi = 36

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 46

In [ ]:
dwi = 46

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 56

In [ ]:
dwi = 56

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 66

In [ ]:
dwi = 66

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 76

In [ ]:
dwi = 76

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 86

In [ ]:
dwi = 86

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)

DWI = 96

In [ ]:
dwi = 96

In [ ]:
# ANN
train_cmd = "--train_subjects "+train_subject+" --model fc1d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model fc1d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 2D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv2d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv2d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
# 3D CNN
train_cmd = "--train_subjects "+train_subject+" --model conv3d --layer 4 --train --label_type A --DWI " + str(dwi)
plot_loss(train_cmd)
for test_subject in test_subjects:
    cmd = '--test_subjects '+test_subject+' --model conv3d --layer 4 --label_type A --DWI ' + str(dwi)
    args = model_parser().parse_args(cmd.split())
    test_model(args)

In [ ]:
visualise0(s02_NDI_img_data)
visualise0(s02_ODI_img_data)
visualise0(s02_FWF_img_data)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'fc1d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv2d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)
visualise4(s02_NDI_img_data, s02_ODI_img_data, s02_FWF_img_data, dwi, 's02_still', 'conv3d', 4, s02_NDI_affine, s02_ODI_affine, s02_FWF_affine)